<a href="https://colab.research.google.com/github/JPinzon91/mtg-deck-builder/blob/main/mtg_deck_builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install mtgsdk

In [ ]:
import requests
#from mtgsdk import Card
import pandas as pd
import json

In [ ]:
deck_csv = pd.read_csv('/content/drive/MyDrive/MTG/El_Dios_Escarabajo.csv',header=0)
default_back = 'https://i.imgur.com/Hg8CwwU.jpeg'

In [ ]:
deck_csv

,card_name,quantity
0,El Dios Escarabajo,1
1,Endless Ranks of the Devil,1
2,Supervisor reluciente,1
3,Command Tower,1
4,Death Baron,1
...,...,...
65,Ciudadela de Nicol Bolas,1
66,Scour All Possibilities,1
67,Mana Geode,1
68,Snow-Covered Swamp,15


In [ ]:
deck_df = pd.DataFrame(columns=['name','language','image_url','back_image'])
deck_token = pd.DataFrame(columns=['name_front','name_back','language','image_url','back_image'])

In [ ]:
h = 100
json_tabletop = {'ObjectStates':[]}
decks_tabletop = {"Name":"DeckCustom",
                 "ContainedObjects":[],
                 "DeckIDs":[],
                 "CustomDeck":{},
                 "Transform":{"posX":0,"posY":1,"posZ":0,"rotX":0,"rotY":180,"rotZ":180,"scaleX":1,"scaleY":1,"scaleZ":1}}

In [ ]:
def card_creator(id,name):
  card_tabletop = {"CardID":id,
                 "Name":"Card",
                 "Nickname":name,
                 "Transform":{"posX":0,"posY":0,"posZ":0,"rotX":0,"rotY":180,"rotZ":180,"scaleX":1,"scaleY":1,"scaleZ":1}}
  return card_tabletop

def card_custom (image_url, back_image):
  custom_card = {"FaceURL":image_url,
                  "BackURL":back_image,
                  "NumHeight":1,
                  "NumWidth":1,
                  "BackIsHidden":True}
  return custom_card

In [ ]:
for i in range(len(deck_csv)):
  #print(deck_csv.iloc[i,0])
  r = requests.get('https://api.scryfall.com/cards/named?fuzzy='+deck_csv.iloc[i,0])
  #Check if card is dual face

  if 'card_faces' in r.json():
    #Check if foreign language is available
    if 'printed_name' in r.json()['card_faces'][0]:
      #Create dual card in token df
      token = {'name_front': r.json()['card_faces'][0]['printed_name'],
               'name_back': r.json()['card_faces'][1]['printed_name'],
               'language':r.json()['lang'],
               'image_url':r.json()['card_faces'][0]['image_uris']['large'],
               'back_image':r.json()['card_faces'][1]['image_uris']['large']}
      #Add token in token deckk
      deck_token.loc[len(deck_token)] = deck_token
      #Create card for main deck
      card = {'name': r.json()['card_faces'][0]['printed_name'],
              'language':r.json()['lang'],
              'image_url':r.json()['card_faces'][0]['image_uris']['large'],
              'back_image':'http://cloud-3.steamusercontent.com/ugc/1647720103762682461/35EF6E87970E2A5D6581E7D96A99F8A575B7A15F/'}

      card_tts = card_creator(0,r.json()['card_faces'][0]['printed_name'])
      custom_card = card_custom(r.json()['card_faces'][0]['image_uris']['large'], default_back)

    else:
      #Create dual card in token df
      token = {'name_front': r.json()['card_faces'][0]['name'],
               'name_back': r.json()['card_faces'][1]['name'],
               'language':r.json()['lang'],
               'image_url':r.json()['card_faces'][0]['image_uris']['large'],
               'back_image':r.json()['card_faces'][1]['image_uris']['large']}
      #Add token in token deckk
      deck_token.loc[len(deck_token)] = deck_token
      #Create card for main deck
      card = {'name':r.json()['card_faces'][0]['name'],
              'language':r.json()['lang'],
              'image_url':r.json()['card_faces'][0]['image_uris']['large'],
              'back_image':'http://cloud-3.steamusercontent.com/ugc/1647720103762682461/35EF6E87970E2A5D6581E7D96A99F8A575B7A15F/'}

      card_tts = card_creator(0,r.json()['card_faces'][0]['name'])
      custom_card = card_custom(r.json()['card_faces'][0]['image_uris']['large'], default_back)
  else:
      #Check if foreign language is available
      if 'printed_name' in r.json():
        card = {'name': r.json()['printed_name'],
                   'language':r.json()['lang'],
                   'image_url':r.json()['image_uris']['large'],
                   'back_image':'http://cloud-3.steamusercontent.com/ugc/1647720103762682461/35EF6E87970E2A5D6581E7D96A99F8A575B7A15F/'}
      #Same but in JSON format
        card_tts = card_creator(0,r.json()['printed_name'])
        custom_card = card_custom(r.json()['image_uris']['large'], default_back)

      else:
        card = {'name': r.json()['name'],
                   'language':r.json()['lang'],
                   'image_url':r.json()['image_uris']['large'],
                   'back_image':'http://cloud-3.steamusercontent.com/ugc/1647720103762682461/35EF6E87970E2A5D6581E7D96A99F8A575B7A15F/'}

        #Same but in JSON format
        card_tts = card_creator(0,r.json()['name'])
        custom_card = card_custom(r.json()['image_uris']['large'], default_back)

  #Add created card in main deck
  for j in range(deck_csv.iloc[i,1]):
    card_tts['CardID'] = h
    decks_tabletop['ContainedObjects'].append(card_tts)
    decks_tabletop['DeckIDs'].append(h)
    decks_tabletop['CustomDeck'][int(h/100)] = custom_card
    deck_df.loc[len(deck_df)] = card
    h = h + 100

In [ ]:
decks_tabletop['Name'] = deck_csv.iloc[0,0]
json_tabletop['ObjectStates'].append(decks_tabletop)

In [ ]:
json_object = pd.

SyntaxError: invalid syntax (<ipython-input-10-195f0d7da939>, line 1)

In [ ]:
with open('/content/drive/MyDrive/MTG/'+deck_csv.iloc[0,0]+'.json','w') as outline:
          json.dump(json_tabletop,outline)


In [ ]:
deck_df
deck_df.to_csv('/content/drive/MyDrive/MTG/'+deck_csv.iloc[0,0]+'.csv', index=False)

In [ ]:
deck_token
deck_token.to_csv('/content/drive/MyDrive/MTG/'+deck_csv.iloc[0,0]+'_token.csv', index=False)